# Programming with Python

## Lecture 04: Walrus operator, pattern matching, and type hints

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 01 Mar, 2025

# New features in latest Python versions

## Walrus operator (`:=`)

The walrus operator (`:=`) in Python is used for assignment expressions, allowing you to assign a value to a variable as part of an expression. Assignment expressions use the walrus operator (`:=`) to both assign and evaluate variable names in a single expression, thus reducing repetition. It was introduced in Python 3.8 [PEP 572](https://peps.python.org/pep-0572/).

### Using in control flow

In [ ]:
# Traditional way

data = input("Enter something: ")
while data != "quit":
    print(f"You entered: {data}")
    data = input("Enter something: ")

In [ ]:
(data = 1) != 3

In [ ]:
# Using Walrus Operator

while (data := input("Enter something: ")) != "quit":
    print(f"You entered: {data}")

In [ ]:
# Traditional way

value = 50
if value > 42:
    print("Your value is greater than 42")

In [ ]:
# Using Walrus Operator

if (value := 50) > 42:
    print("Your value is greater than 42")

### Using in list comprehensions

You can use `:=` to avoid redundant calculations inside list comprehensions.

In [ ]:
def expensive_computation(x):
    return x ** 2

In [ ]:
# Traditional way

nums = [1, 2, 3, 4, 5]

squares = [expensive_computation(x) for x in nums if expensive_computation(x) > 10]
squares

In [ ]:
# Using Walrus Operator

squares = [sq for x in nums if (sq := expensive_computation(x)) > 10]
squares

## Structural pattern matching

Structural pattern matching was introduced in Python 3.10 [PEP 636](https://peps.python.org/pep-0636/) with the `match` statement. It allows for concise, readable, and expressive ways to handle complex conditional logic, similar to switch-case in other languages but with much more power.

```python
match subject:
    case pattern1:
        # code block for pattern1
    case pattern2:
        # code block for pattern2
    # ...
    case _:
        # code block for the default case (like 'else')
```

- `subject` is the variable you are matching against.
- `case` specifies a pattern to match against the subject.
- `_` is used for the default case, matching anything if no other pattern matches.

We can also use `if` guard conditions with cases for more complex logic.

### Basic usage

In [ ]:
def check_number(n):
    match n:
        case 1 | 2:
            return "One or two"
        case 3 | 4 | 5:
            return "Three, four or five"
        case _:
            return "Something else"

print(check_number(1))
print(check_number(2))
print(check_number(4))
print(check_number(6))

### Matching tuples & lists

In [ ]:
def process_tuple(data):
    match data:
        case (1, x):
            return f"Starts with 1, second element is {x}"
        case (x, y, z):
            return f"Three-element tuple: {x}, {y}, {z}"
        case _:
            return "Something else"

print(process_tuple((1, 100)))
print(process_tuple((5, 10, 15)))

This shows that:

- Patterns can bind variables (e.g. `x`, `y`, `z`).
- If a tuple of two elements is passed, it matches the first case. If a tuple of three elements is passed, it matches the second case.

In [ ]:
def process_list(numbers):
    match numbers:
        case []:
            return "No numbers provided"
        case [first]:
            return f"Only one number: {first}"
        case [first, second]:
            return f"Two numbers: {first} and {second}"
        case [first, *rest]:
            return f"First number: {first}, and the rest: {rest}"
        case _:
            return "Not a list of numbers"

In [ ]:
print(process_list([10, 20, 30]))
print(process_list([]))
print(process_list(34))

In this example, different patterns are used to match lists of various lengths, including using the `*rest` syntax to capture the remainder of the list.

### Matching dictionaries

In [ ]:
def process_dict(data):
    match data:
        case {"name": name, "age": age}:
            return f"Name: {name}, Age: {age}"
        case {"name": name}:
            return f"Name: {name}"
        case {"city": city, **rest}:
            return f"City: {city}, Other details: {rest}"
        case _:
            return "Unknown format"

print(process_dict({"name": "Alice", "age": 30}))
print(process_dict({"name": "Alice"}))
print(process_dict({"city": "London", "salary": 100_000}))

### Matching classes (object matching)

You can use pattern matching with custom objects by defining attributes. 

Also, `if` statements allow filtering matches.

In [ ]:
from dataclasses import dataclass


@dataclass
class Vehicle:
    pass


@dataclass
class Car(Vehicle):
    make: str
    model: str


@dataclass
class Truck(Vehicle):
    make: str
    towing_capacity: int


def describe_vehicle(vehicle):
    match vehicle:
        case Car(make="Tesla", model=model):
            return f"A Tesla car, model {model}"
        case Car():
            return "A car of some make and model"
        case Truck(towing_capacity=towing) if towing > 5000:
            return "A heavy-duty truck"
        case _:
            return "Some type of vehicle"

In [ ]:
car = Car("Tesla", "Cybertruck")
print(describe_vehicle(car))

In [ ]:
car = Car("Porsche", "911")
print(describe_vehicle(car))

In [ ]:
truck = Truck("Ford", 5500)
print(describe_vehicle(truck))

In [ ]:
truck = Truck("Ford", 4200)
print(describe_vehicle(truck))

### Positive number validation

The following `PositiveNumber` class uses structural pattern matching to check number types.

In [ ]:
class PositiveNumber:
    def __init__(self, value):
        match value:
            case int() | float() if value > 0:
                self.value = float(value)
            case int() | float():
                raise ValueError("number must be positive")
            case complex():
                raise TypeError("number must be real")
            case _:
                raise TypeError("value must be a number")

    def __repr__(self):
        return f"PositiveNumber({self.value!r})"

In [ ]:
PositiveNumber(42)

In [ ]:
PositiveNumber(3.14)

In [ ]:
PositiveNumber(0)

In [ ]:
PositiveNumber(4 + 2j)

In [ ]:
PositiveNumber("fourty two")

# Type hints

## Dynamic typing

Python is **dynamically typed**, meaning variable types are determined at runtime rather than being explicitly declared. This provides flexibility but can lead to runtime errors if types are misused.

In [ ]:
x = 10      # x is an int
x = "Hello" # Now x is a str (type changes dynamically)

In [ ]:
def add(a, b):
    return a + b

print(add(2, 3))
print(add("Hi ", "Joe"))

In [ ]:
data = [1, "apple", 3.14, True]  # List with multiple types

## Static typing

**Static typing** means that variable types are explicitly declared and checked at compile time rather than at runtime. This ensures type safety and prevents type-related errors before execution.

```c
int x = 10;
```

## Gradual typing

**Gradual typing** in Python allows developers to introduce type hints incrementally while maintaining the flexibility of dynamic typing. This is implemented through type hints (introduced in [PEP 484](https://peps.python.org/pep-0484/)) and enforced optionally using tools like `mypy`.

- Python is dynamically typed, meaning variable types are determined at runtime.
- With **gradual typing**, you can add **optional** type hints to improve code readability, catch errors early, and enhance tooling support.
- Type hints **do not** change how Python executes the code; they serve as annotations for static analysis.

## `mypy`

`mypy` is a static type checker for Python that helps enforce type hints at compile time rather than runtime. It ensures that your Python code adheres to specified type annotations, catching potential bugs before execution.

In [ ]:
!pip install mypy

### Usage

By default, `mypy` ignores function signatures with no annotations.

```sh
mypy script.py
```

Argument `--disallow-incomplete-defs` can be passed to disallow defining functions with incomplete type annotations.

```sh
mypy --disallow-incomplete-defs script.py
```

## Practice

Show examples 1, 2, and 3.

## Duck typing in Python

**Duck typing** is a concept in dynamic typing where the type of an object is determined by its behavior (methods and properties) rather than its explicit class.

> "If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck."

### Benefits

- More flexible code (no need to check types explicitly).
- Encourages polymorphism without requiring inheritance.
- Reduces boilerplate code by avoiding type checks (`isinstance()` or `type()`).

### Drawbacks

- May cause runtime errors if an object does not have the expected method.
- Lack of explicit type definitions.

In [ ]:
class Duck:
    def speak(self):
        return "Quack!"

class Dog:
    def speak(self):
        return "Woof!"

def make_sound(animal):
    return animal.speak()  # No type checking

duck = Duck()
dog = Dog()

print(make_sound(duck))
print(make_sound(dog))

## Nominal typing

**Nominal typing** means that type compatibility is based on explicit class inheritance or declared type names, rather than just structure or behavior (as in duck typing).

In nominal typing, an object is considered of a certain type only if it is an instance of a declared class or explicitly inherits from it.

## Practice

Show example 4.

## `Any` type

In Python's gradual typing system, `Any` is a special type that allows a variable to hold any data type. `Any` is a magic type that sits at the top and the bottom of the type hierarchy. It’s simultaneously the most general type—so that an argument `n: Any` accepts values of every type—and the most specialized type, supporting every possible operation.

## Practice

Show example 5.

## Consistent-with

Let `T2` is a **subtype-of** `T1`, meaning the following:

```python
class T1:
    pass

class T2(T1):
    pass
```

In a gradual type system, there is another relationship: **consistent-with**, which applies wherever **subtype-of applies**, with special provisions for type `Any`. The rules for **consistent-with** are: 

1. Given `T1` and a subtype `T2`, then `T2` is consistent-with `T1` (Liskov substitution).
2. Every type is consistent-with `Any`: you can pass objects of every type to an argument declared of type `Any`.
3. `Any` is consistent-with every type: you can always pass an object of type `Any` where an argument of another type is expected.

## Simple types and classes

Simple types, such as `int`, `float`, `str`, can be used as type hints. Also, classes from the standard library and defined by the user can be used in type hints.

In general, subclasses are consistent-with their superclasses.

Also, the following holds true:

- `int` is consistent-with `float`.
- `float` is consistent-with `complex`.
- `int` is consistent-with `complex`.

## Union Type

`Union` is used to specify that a variable or function parameter can be multiple types.

## Practice

Show example 6.

## Generic collections

The following list shows collections that use the simplest form of generic type hint, `container[item]`, meaning it is a `container` collection of `item` objects. 


- `list`
- `set`
- `frozenset`
- `collections.deque`
- `abc.Container`
- `abc.Collection`
- `abc.Sequence`
- `abc.MutableSequence`
- `abc.Set`
- `abc.MutableSet`

### Practice

Show example 7.

## Tuple types



### Practice

Show example 8.